In [2]:
import gc

import duckdb
from transformers import BigBirdModel, BigBirdTokenizerFast

In [3]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus3.duckdb', read_only=False)

In [3]:
con.execute("CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./data/202202-stj/metadadosPublicacao202202.json');")

CatalogException: Catalog Error: Table with name "metadadosPublicacao202202" already exists!

In [ ]:
con.execute("DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;")

In [4]:
con.execute("select * from metadadosPublicacao202202").df()

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
0,144948780,1644458400000,ACÓRDÃO,202002151590,REsp 1890871,1598238000000,1598929200000,ASSUSETE MAGALHÃES,None,Concedendo,None,10254;10225;10254;10225
1,145620297,1645412400000,ACÓRDÃO,202103074377,HC 695900,1632279600000,1632279600000,SEBASTIÃO REIS JÚNIOR,None,Concedendo,None,3372;3372
2,139579332,1644890400000,ACÓRDÃO,202103073420,HC 695866,1632279600000,1632279600000,ANTONIO SALDANHA PALHEIRO,None,Concedendo,None,4355;4355;3608
3,141816757,1645758000000,ACÓRDÃO,202103076854,Rcl 42334,1632279600000,1632452400000,REYNALDO SOARES DA FONSECA,None,Concedendo,None,3608;3608
4,144148942,1645412400000,ACÓRDÃO,202103167750,HC 697730,1632884400000,1633057200000,OLINDO MENEZES (DESEMBARGADOR CONVOCADO DO TRF...,None,Concedendo,None,3608;3608
...,...,...,...,...,...,...,...,...,...,...,...,...
35399,144531393,1643940000000,DECISÃO,201100020306,REsp 1228590,1294192800000,1296007200000,OG FERNANDES,EMBARGOS DE DECLARAÇÃO,Concedendo,Embargos de Declaração Acolhidos,10318
35400,143092827,1645758000000,DECISÃO,201001614248,REsp 1210458,1285642800000,1288058400000,OG FERNANDES,None,Concedendo,Conhecido em parte o recurso e provido em parte,7760
35401,142924179,1643767200000,DECISÃO,200902062079,EREsp 1163268,1256004000000,1256608800000,FRANCISCO FALCÃO,None,Concedendo,Conhecido o recurso e provido,10028;10028;1000238
35402,145440916,1644976800000,DECISÃO,200801470759,AR 3998,1214794800000,1214794800000,BENEDITO GONÇALVES,EMBARGOS DE DECLARAÇÃO,Concedendo,Embargos de Declaração Acolhidos,8826


In [4]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_description (
    ID INT PRIMARY KEY,
    content TEXT,
    summarize TEXT
  );
""")

CatalogException: Catalog Error: Table with name "decisions_description" already exists!

In [8]:
con.execute("COPY decisions_description FROM './decisions_description_202310102238.csv' DELIMITER ',' HEADER;")

In [6]:
con.execute("ALTER TABLE decisions_description DROP COLUMN summarize;")

In [5]:
con.execute("select * from embedding").df()

,id,data
0,121184492,"[0.19962230324745178, 0.2647660970687866, -0.3..."
1,122027196,"[0.10591266304254532, 0.2007523626089096, -0.3..."
2,123312824,"[0.15172050893306732, 0.16319796442985535, -0...."
3,123447928,"[0.15098059177398682, 0.37319862842559814, -0...."


In [11]:
con.execute("CREATE TABLE embedding(id INT UNIQUE PRIMARY KEY, data FLOAT8[]);")

CatalogException: Catalog Error: Table with name "embedding" already exists!

In [45]:
jus2_embedding = con.execute("select * from BigBird").df()

con.register('df_temp', jus2_embedding)
con.execute("INSERT INTO embedding SELECT * FROM df_temp")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
con.execute("drop table embedding")

In [13]:
con.execute("""
CREATE TABLE embedding_norm(
    id INT UNIQUE REFERENCES embedding(id),
    norm FLOAT8
);
""")

In [12]:
con.execute("CREATE INDEX embedding_norm_index ON embedding_norm (norm);")

In [14]:
con.execute("drop table embedding_norm")

In [4]:
tokenizer = BigBirdTokenizerFast.from_pretrained(
    "google/bigbird-roberta-base")  # large
model = BigBirdModel.from_pretrained("google/bigbird-roberta-base",
                                     attention_type="original_full"
                                     #attention_type="block_sparse"
                                     )

In [5]:
decisions_description = con.execute("SELECT ID,content FROM decisions_description").df()
decisions_description

,ID,content
0,121184492,DECISION > It is treated of habeas corpus impe...
1,122027196,DECISION<br> It is considered a special appeal...
2,123312824,DECISION<br> It is considered an ordinary appe...
3,123447928,DECISION<br> It is treated of habeas corpus im...
4,123917739,DECISION<br> It is considered a special appeal...
...,...,...
5800,146076514,DECISION<br> By understanding that better exam...
5801,144454767,DECISION<br> By understanding that better exam...
5802,145089835,DECISION<br> By understanding that better exam...
5803,145511648,DECISION<br> By understanding that better exam...


In [ ]:
import spacy
nlp = spacy.load('pt_core_news_lg')

In [7]:
from tqdm import tqdm

for index, content in tqdm(enumerate(decisions_description["content"])):
  if index >= 4 and index <= 100:
    out_str = content
    seqDocumento = int(decisions_description["ID"][index])
    
    inputs = tokenizer(out_str, return_tensors="pt")
    outputs = model(**inputs)

    print(index,seqDocumento,len(content),len(out_str),outputs.last_hidden_state[0].shape)

    vetor = outputs.last_hidden_state[0].flatten().detach().numpy()
    con.execute('INSERT INTO embedding (id,data) VALUES (?,?)', (seqDocumento,vetor))

con.commit()

0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (5724 > 4096). Running this sequence through the model will result in indexing errors
4it [00:00, 46.44it/s]


RuntimeError: The expanded size of the tensor (5724) must match the existing size (4096) at non-singleton dimension 1.  Target sizes: [1, 5724].  Tensor sizes: [1, 4096]

In [17]:
con.execute("SELECT * FROM embedding_norm").df()

,id,norm
0,127506614,344.751405
1,128070511,104.631807
2,130072596,212.710395
3,130527099,125.903486
4,131460066,236.974861
...,...,...
190,140050588,172.508452
191,140477975,190.283220
192,140508284,150.179740
193,140527996,143.797050


In [16]:
con.execute("""
WITH norms AS (
    SELECT 
        id,
        SQRT(SUM(POWER(val, 2))) AS norm
    FROM 
    (
        SELECT
            id,
            UNNEST(data) AS val
        FROM 
            embedding
    ) AS subquery
    GROUP BY id
)

INSERT INTO embedding_norm (id, norm)
SELECT id, norm
FROM norms
ON CONFLICT (id) 
DO UPDATE SET norm = EXCLUDED.norm;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
con.execute("""
WITH DotProducts AS (
    WITH expanded AS (
        WITH norm_prox AS (
          (SELECT id, norm
          FROM embedding_norm
          WHERE norm < 42
          ORDER BY norm DESC
          LIMIT 128)

          UNION ALL

          (SELECT id, norm
          FROM embedding_norm
          WHERE norm > 42
          ORDER BY norm ASC
          LIMIT 128)
        )
        SELECT
            a.id AS idA,
            b.id AS idB,
            unnest(a_data.data) AS a_data,
            unnest(b_data.data) AS b_data
        FROM
            norm_prox a
        INNER JOIN
            norm_prox b ON a.id < b.id
        JOIN embedding a_data ON a.id = a_data.id
        JOIN embedding b_data ON b.id = b_data.id
    )
    SELECT
        idA,
        idB,
        sum(a_data * b_data) AS dot_product
    FROM
        expanded
    GROUP BY
        idA, idB
),
CosineSimilarities AS (
    SELECT 
        d.idA,
        d.idB,
        CASE 
            WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
            ELSE d.dot_product / (n1.norm * n2.norm)
        END AS cosine_similarity            
    FROM 
        DotProducts d
    JOIN 
        embedding_norm n1 ON d.idA = n1.id
    JOIN 
        embedding_norm n2 ON d.idB = n2.id
    WHERE 
        d.idA < d.idB
)
SELECT 
    idA,
    idB,
    cosine_similarity
FROM 
    CosineSimilarities
WHERE 
    cosine_similarity > 0.7;
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,idA,idB,cosine_similarity
0,137883071,138060974,1.0


In [19]:
gc.collect()

738

In [20]:
con.commit()

In [58]:
con.commit()
con.close()